In [1]:
# import dependencies 
import pandas as pd
import datetime as dt
from datetime import datetime
import numpy as np
from config import pg_password
from sqlalchemy import create_engine

## Read in data using pd.read_csv()

In [2]:
# Read in csv
covid_data_df = pd.read_csv(r"us-counties.csv")
covid_data_df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


## Check DataFrame values
- Total date values
- Earliest date
- Latest date
    

In [3]:
# Identify total date nunique() values
covid_data_df['date'].nunique()

369

In [4]:
# Identify min() date range
covid_data_df['date'].min()

'2020-01-21'

In [5]:
# Identify max() date range
covid_data_df['date'].max()

'2021-01-23'

## Clean and Transform Data
- Format date from string to_datetime
- Gather data for only California
- Gather data for only 2020

In [6]:
# convert date column from string to datetime
covid_data_df['date'] = pd.to_datetime(covid_data_df['date'], format='%Y/%m/%d')
covid_data_df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [7]:
# Create dataFrame for only California 
covid_cali_df = covid_data_df.loc[covid_data_df["state"] == "California"] 
covid_cali_df.head()

,date,county,state,fips,cases,deaths
5,2020-01-25,Orange,California,6059.0,1,0.0
9,2020-01-26,Los Angeles,California,6037.0,1,0.0
10,2020-01-26,Orange,California,6059.0,1,0.0
14,2020-01-27,Los Angeles,California,6037.0,1,0.0
15,2020-01-27,Orange,California,6059.0,1,0.0


In [8]:
# Get county names 
covid_cali_df.county.unique()

array(['Orange', 'Los Angeles', 'Santa Clara', 'San Francisco',
       'San Diego', 'Humboldt', 'Sacramento', 'Solano', 'Marin', 'Napa',
       'Sonoma', 'Alameda', 'Placer', 'San Mateo', 'Contra Costa', 'Yolo',
       'Fresno', 'Madera', 'Riverside', 'Santa Cruz', 'Shasta',
       'San Joaquin', 'Ventura', 'Stanislaus', 'Tulare', 'San Benito',
       'San Luis Obispo', 'San Bernardino', 'Santa Barbara', 'Nevada',
       'Kern', 'Monterey', 'Mendocino', 'Amador', 'Imperial', 'Butte',
       'El Dorado', 'Siskiyou', 'Yuba', 'Unknown', 'Calaveras', 'Merced',
       'Mono', 'Inyo', 'Sutter', 'Colusa', 'Kings', 'Glenn', 'Tuolumne',
       'Alpine', 'Plumas', 'Del Norte', 'Tehama', 'Lake', 'Mariposa',
       'Trinity', 'Sierra', 'Lassen', 'Modoc'], dtype=object)

In [9]:
# Clean data -  remove 'unknown'
clean_covid_df = covid_cali_df[covid_cali_df.county != 'Unknown']
clean_covid_df.head()

,date,county,state,fips,cases,deaths
5,2020-01-25,Orange,California,6059.0,1,0.0
9,2020-01-26,Los Angeles,California,6037.0,1,0.0
10,2020-01-26,Orange,California,6059.0,1,0.0
14,2020-01-27,Los Angeles,California,6037.0,1,0.0
15,2020-01-27,Orange,California,6059.0,1,0.0


In [10]:
# # Confirm that only 58 counties in California
clean_covid_df.county.nunique()

58

In [11]:
covid_cali_2020_df = clean_covid_df.loc[clean_covid_df["date"] < "2021-01-01"]
covid_cali_2020_df.head()

,date,county,state,fips,cases,deaths
5,2020-01-25,Orange,California,6059.0,1,0.0
9,2020-01-26,Los Angeles,California,6037.0,1,0.0
10,2020-01-26,Orange,California,6059.0,1,0.0
14,2020-01-27,Los Angeles,California,6037.0,1,0.0
15,2020-01-27,Orange,California,6059.0,1,0.0


In [12]:
# Identify date total nunique() total
covid_cali_2020_df['date'].nunique()

342

In [13]:
# Identify earliest date
covid_cali_2020_df['date'].min()

Timestamp('2020-01-25 00:00:00')

In [14]:
# Identify max() date to confirm it reflects 12-31-2020
covid_cali_2020_df['date'].max()

Timestamp('2020-12-31 00:00:00')

## Create DataFrames for specific viualization needs
- cali only DataFrames grouped by date with cases total 
- socal only DataFrames grouped by county with cases total
- nocal only DataFrames grouped by county wth cases total

In [15]:
# group all of CA data by date
cali_groupby_date_cases_df = covid_cali_2020_df.groupby('date')

In [16]:
# Get sum for cases by date
cali_date_total = cali_groupby_date_cases_df['cases'].sum()
# Confirm data is correct
cali_date_total.head()

date
2020-01-25    1
2020-01-26    2
2020-01-27    2
2020-01-28    2
2020-01-29    2
Name: cases, dtype: int64

In [17]:
# Create series into DataFrame
cali_date_total_df = pd.DataFrame(cali_date_total)
cali_date_total_df.head()

,cases
date,
2020-01-25,1
2020-01-26,2
2020-01-27,2
2020-01-28,2
2020-01-29,2


In [18]:
# Identify total day count
cali_date_total_df.count()

cases    342
dtype: int64

In [19]:
# Create socalArray for dataFrame with socal counties only
socalArray = ["Inyo", "Kern", "San Luis Obispo", "Santa Barbara", "Ventura", "Los Angeles", "San Bernardino", "Orange", "Riverside", "San Diego", "Imperial"]

# Creating only socal_data_df by selecting socal counties with isin
socal_data_df = clean_covid_df.loc[clean_covid_df["county"].isin(socalArray)]
socal_data_df.head()

,date,county,state,fips,cases,deaths
5,2020-01-25,Orange,California,6059.0,1,0.0
9,2020-01-26,Los Angeles,California,6037.0,1,0.0
10,2020-01-26,Orange,California,6059.0,1,0.0
14,2020-01-27,Los Angeles,California,6037.0,1,0.0
15,2020-01-27,Orange,California,6059.0,1,0.0


In [20]:
# Confirm that only the 11 socal counties are in dataFrame
socal_data_df.nunique()

date       365
county      11
state        1
fips        11
cases     2863
deaths    1146
dtype: int64

In [21]:
# Grouped socal by county
grouped_socal_counties = socal_data_df.groupby('county')

In [22]:
# Get sum for cases by county
grouped_socal_county_totals =  grouped_socal_counties['cases'].sum()
grouped_socal_county_totals.head()

county
Imperial        3043737
Inyo              56686
Kern            7884109
Los Angeles    81587695
Orange         16437080
Name: cases, dtype: int64

In [23]:
# Create DataFrame from series
grouped_socal_county_totals_df = pd.DataFrame(grouped_socal_county_totals)
grouped_socal_county_totals_df.head()

,cases
county,
Imperial,3043737
Inyo,56686
Kern,7884109
Los Angeles,81587695
Orange,16437080


In [24]:
# Using socal_data_df, merge with clean_covid_cali_df and create duplicates for socal counties
duplicates = pd.merge(clean_covid_df, socal_data_df, how= 'inner', left_on=['date', 'county'], right_on=['date', 'county'], left_index=True)

# Use .drop duplicates on index to create nocal_data_df
norcal_data_df = clean_covid_df.drop(duplicates.index)
norcal_data_df.head()

,date,county,state,fips,cases,deaths
36,2020-01-31,Santa Clara,California,6085.0,1,0.0
42,2020-02-01,Santa Clara,California,6085.0,1,0.0
49,2020-02-02,San Francisco,California,6075.0,2,0.0
50,2020-02-02,Santa Clara,California,6085.0,2,0.0
57,2020-02-03,San Francisco,California,6075.0,2,0.0


In [25]:
# Confirm that only 47 counties for nocal
norcal_data_df.nunique()

date       359
county      47
state        1
fips        47
cases     5721
deaths     641
dtype: int64

In [26]:
# Grouped norcal data by county
grouped_norcal_county_df = norcal_data_df.groupby('county')

In [27]:
# Get cases sum() by county 
grouped_norcal_county_total = grouped_norcal_county_df['cases'].sum()
grouped_norcal_county_total.head()

county
Alameda      5741871
Alpine          4436
Amador        150719
Butte         716705
Calaveras      99176
Name: cases, dtype: int64

In [28]:
# Create DataFrame from series
grouped_norcal_county_total_df = pd.DataFrame(grouped_norcal_county_total)
grouped_norcal_county_total_df.head()

,cases
county,
Alameda,5741871
Alpine,4436
Amador,150719
Butte,716705
Calaveras,99176


## Make connection to Postgres

In [29]:
clean_covid_df.dtypes

date      datetime64[ns]
county            object
state             object
fips             float64
cases              int64
deaths           float64
dtype: object

In [30]:
# Make connection to telecommunication_db in postgresql
connection_string = f"postgres:{pg_password}@localhost:5432/covid_mask_effect_db"
engine = create_engine(f'postgresql://{connection_string}')

In [31]:
# Read in tables_names() from postgres
engine.table_names()

['socal_data', 'cali_data', 'nocal_data']

## Load DataFrames to Postgres

In [387]:
# cali_date_total_df.to_sql(name='cali_data', con=engine, if_exists='append', index=True)

In [388]:
# grouped_socal_county_totals_df.to_sql(name='socal_data', con=engine, if_exists='append', index=True)

In [32]:
# grouped_norcal_county_total_df.to_sql(name='nocal_data', con=engine, if_exists='append', index=True)

## Read in tables from pgadmin to confirm data was received correclty

In [33]:
pd.read_sql_query('SELECT * FROM cali_data', con=engine).head()

,date,cases
0,2020-01-25,1
1,2020-01-26,2
2,2020-01-27,2
3,2020-01-28,2
4,2020-01-29,2


In [34]:
pd.read_sql_query('SELECT * FROM socal_data', con=engine).head()

,county,cases
0,Imperial,3043737
1,Inyo,56686
2,Kern,7884109
3,Los Angeles,81587695
4,Orange,16437080


In [35]:
pd.read_sql_query('SELECT * FROM nocal_data', con=engine).head()

,county,cases
0,Alameda,5741871
1,Alpine,4436
2,Amador,150719
3,Butte,716705
4,Calaveras,99176


# Convert new tables into csv

In [36]:
covid_cali_2020_df.to_csv('output/covid_cali_2020_df.csv')

In [37]:
cali_date_total_df.to_csv('output/cali_date_total_df.csv')

In [38]:
grouped_socal_county_totals_df.to_csv('output/grouped_socal_county_totals_df.csv')

In [40]:
grouped_norcal_county_total_df.to_csv('output/grouped_norcal_county_total_df.csv')